In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

data = pd.read_csv('../input/happy-whale-and-dolphin/train.csv')

classes = data['individual_id'].unique()

print(len(data['individual_id'].unique()))
#print(data.shape)

In [ ]:
import os 
print(len(os.listdir('../input/happy-whale-and-dolphin/train_images')))

In [ ]:
#making folders for each label

os.mkdir('/kaggle/data')
os.mkdir('/kaggle/data/train_data')
for i in classes:
    s = '/kaggle/data/train_data/'+str(i)
    os.mkdir(s)
        

In [ ]:
import cv2
import matplotlib.pyplot as plt
img = cv2.imread('../input/happy-whale-and-dolphin/test_images/000fe6ebfc9893.jpg')
plt.imshow(img)
print(img.shape)

In [ ]:
import shutil

for i in range(len(data)):
    #print(data.iloc[i])
    s1 = '../input/happy-whale-and-dolphin/train_images'+'/'+data.iloc[i]['image']
    s2 = '/kaggle/data/train_data/'+data.iloc[i]['individual_id']+'/'+data.iloc[i]['image']
    shutil.copy(s1,s2)
    

In [ ]:
for i in os.listdir('/kaggle/data/train_data/'):
    print(os.listdir('/kaggle/data/train_data/f8ae3dfb8f11'))
    break

In [ ]:
data.loc[data['image'] == '64b52ad21c7fb9.jpg']['individual_id']

In [ ]:
import pandas as pd
import numpy as np
import shutil
import tensorflow as tf
from tensorflow import keras
from keras import Sequential, Model
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Activation,MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [ ]:
base_model = tf.keras.applications.xception.Xception(weights='imagenet')
base_model.summary()

In [ ]:
x = base_model.layers[-2].output
#pred = Dense(units=4096, activation='relu')(x)
pred1 = Dense(units=15587, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=pred1)
model.summary()

In [ ]:
for layer in model.layers[:-2]:
    layer.trainable = False
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ReduceLROnPlateau = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor= 0.1, min_lr=0.00001)

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=keras.applications.xception.preprocess_input).flow_from_directory(directory='/kaggle/data/train_data',target_size=(299,299) , batch_size=100, interpolation='bilinear')
valid_batches = ImageDataGenerator(preprocessing_function=keras.applications.xception.preprocess_input).flow_from_directory(directory='/kaggle/data/train_data',target_size=(299,299) , batch_size=100, interpolation='bilinear')

In [ ]:
model.fit_generator(train_batches, epochs=5, callbacks=[ReduceLROnPlateau], steps_per_epoch=510, validation_data = valid_batches, validation_steps=100)

In [ ]:
model.save('model1.h5')

In [ ]:
<a href="./model1.h5"> Download File </a>

In [ ]:
model1 = keras.models.load_model('../input/not-so-happy-whales/model1.h5')

In [ ]:
model1.evaluate(valid_batches, steps = 100)

In [ ]:
test_batches = ImageDataGenerator(preprocessing_function=keras.applications.xception.preprocess_input).flow_from_directory(directory='../input/happy-whale-and-dolphin/',classes=['test_images'],target_size=(299,299) , interpolation='bilinear', shuffle=False, batch_size=10)

In [ ]:
test_batches.reset()
y_pred=model1.predict_generator(test_batches,verbose=2,steps=2796)

In [ ]:
print(type(y_pred))

In [ ]:
np.savetxt('pred.csv',y_pred)